# Pytorch

In [1]:
import torch

In [18]:
tensor0 = torch.zeros(1, 5)
tensor1 = torch.range(1, 5).view(1, -1)
tensor2 = (torch.range(1, 15)*10).view(3,-1)
print(tensor0)
print(tensor1)
print(tensor2)

tensor([[0., 0., 0., 0., 0.]])
tensor([[1., 2., 3., 4., 5.]])
tensor([[ 10.,  20.,  30.,  40.,  50.],
        [ 60.,  70.,  80.,  90., 100.],
        [110., 120., 130., 140., 150.]])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
print(f"{tensor1}\n" \
      f"{tensor2}")

tensor([[1., 2., 3., 4., 5.]])
tensor([[ 10.,  20.,  30.,  40.,  50.],
        [ 60.,  70.,  80.,  90., 100.],
        [110., 120., 130., 140., 150.]])


In [43]:
# result = torch.matmul(tensor2, tensor1.t())
result = torch.mm(tensor1, tensor2.t())
print(result)
print(result.sum())

tensor([[ 550., 1300., 2050.]])
tensor(3900.)


In [36]:
tmp = torch.dot(tensor1.view(-1), tensor2[0].view(-1))
print(tmp)

tensor(550.)


In [65]:
tensor3 = (torch.range(1, 30)).view(-1, 5)
print(tensor3)

tensor([[ 1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10.],
        [11., 12., 13., 14., 15.],
        [16., 17., 18., 19., 20.],
        [21., 22., 23., 24., 25.],
        [26., 27., 28., 29., 30.]])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


In [108]:
%%time
#%%timeit -r 100

result = None
n = tensor3.shape[0]
# print(f"Dim : {n}")

for i in range(n):
    if result is None:
        result = torch.mm(tensor3[i].view(1, -1), tensor3[i+1:n].t()).sum()
        # print(result)
    else:
        result += torch.mm(tensor3[i].view(1, -1), tensor3[i+1:n].t()).sum()
        # print(result)
        
# print(result)

CPU times: user 306 µs, sys: 0 ns, total: 306 µs
Wall time: 318 µs


In [104]:
%%time 
#%%timeit -r 100

result = 0
n = tensor3.shape[0]
# print(f"Dim : {n}")

for i in range(n):
    result += torch.mm(tensor3[i].view(1, -1), tensor3[i+1:n].t()).sum()
    # print(result)
        
# print(result)

88.6 µs ± 1.81 µs per loop (mean ± std. dev. of 100 runs, 10000 loops each)


In [87]:
%%time

result = None
n = tensor3.shape[0]
print(f"Dim : {n}")

for i in range(n):
    for j in range(i+1, n):
        if result is None:
            result = torch.dot(tensor3[i], tensor3[j])
            print(result)
        else:
            result += torch.dot(tensor3[i], tensor3[j])
            print(result)
        
print(result)

Dim : 6
tensor(130.)
tensor(335.)
tensor(615.)
tensor(970.)
tensor(1400.)
tensor(1930.)
tensor(2660.)
tensor(3590.)
tensor(4720.)
tensor(5900.)
tensor(7405.)
tensor(9235.)
tensor(11315.)
tensor(13845.)
tensor(17075.)
tensor(17075.)
CPU times: user 8.46 ms, sys: 88 µs, total: 8.55 ms
Wall time: 7.23 ms


# Multi-process

In [2]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def busy_cpu(index):
    return index**2

futures = []
worker_num = 4
task_num = 8

with ProcessPoolExecutor(worker_num) as executer:
    for i in range(task_num):
        futures.append(executer.submit(busy_cpu, i))
        
for x in as_completed(futures):
    print(x.result())

49
16
0
36
1
9
25
4


In [8]:
%%time

import multiprocessing

def busy_cpu(index):
    return index**2

futures = []
task_num = int(1e+8)

for i in range(task_num):
    futures.append(i)
        
process = max(1, multiprocessing.cpu_count()-1)
p = multiprocessing.Pool(process)
result = p.map(busy_cpu, futures)

print(result[0], result[-1])

0 9999999800000001
CPU times: user 17.1 s, sys: 4.64 s, total: 21.8 s
Wall time: 21.7 s


In [7]:
%%time

def busy_cpu(index):
    return index**2

futures = []
worker_num = 4
task_num = int(1e+8)

for i in range(task_num):
    futures.append(busy_cpu(i))
        
print(futures[0], futures[-1])

0 9999999800000001
CPU times: user 31.2 s, sys: 943 ms, total: 32.1 s
Wall time: 32 s
